In [1]:
import pandas as pd
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import numpy as np

#  Cleaning the data

- Import the data

- Create the labels

- Filter the data to only include fields we want

- Drop companies with <2000 MM

- Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month

In [2]:
data2014 = pd.read_csv("Our_Dataset/2014dataset.csv")
data2015 = pd.read_csv("Our_Dataset/2015dataset.csv")
data2016 = pd.read_csv("Our_Dataset/2016dataset.csv")
data2017 = pd.read_csv("Our_Dataset/2017dataset.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,52,323,649,882,932,1092,1202) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,57,204,341,376,377,396,430,431,928,944,1006,1013,1151,1160,1169,1179,1411) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,17,163,355,406,702,835,847,1083,1156,1183,1185,1200) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create the labels.
# By index:
# label 2 is transfers
# label 3 is premiums
# label 4 is costs
# label 5 is member months
# label 6 is reinsurance (individual group only)

ind_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                  "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
                  "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                  "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)"] # no more reinsurance as of 2017
ind_labels_15 = [label.replace('15', '16') for label in ind_labels_14]
ind_labels_16 = [label.replace('15', '17') for label in ind_labels_14]
ind_labels_17 = [label.replace('15', '18') for label in ind_labels_14][:-1]

sg_labels_14 = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME",
                 "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)",
                  "1.1Directpremiumwritten7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "2.16Totalincurredclaims7HealthInsuranceSMALLGROUPTotalasof3/31/15",
                  "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"]
sg_labels_15 = [label.replace('15', '16') for label in sg_labels_14]
sg_labels_16 = [label.replace('15', '17') for label in sg_labels_14]
sg_labels_17 = [label.replace('15', '18') for label in sg_labels_14]

In [4]:
# Filter the data to include only the fields we want

ind_data_14 = data2014[ind_labels_14].dropna()
ind_data_15 = data2015[ind_labels_15].dropna()
ind_data_16 = data2016[ind_labels_16].dropna()
ind_data_17 = data2017[ind_labels_17].dropna()
sg_data_14 = data2014[sg_labels_14].dropna()
sg_data_15 = data2015[sg_labels_15].dropna()
sg_data_16 = data2016[sg_labels_16].dropna()
sg_data_17 = data2017[sg_labels_17].dropna()

In [5]:
# Drop companies with <2000 MM
ind_data_14 = ind_data_14.where(ind_data_14[ind_labels_14[5]] > 2000).dropna()
ind_data_15 = ind_data_15.where(ind_data_15[ind_labels_15[5]] > 2000).dropna()
ind_data_16 = ind_data_16.where(ind_data_16[ind_labels_16[5]] > 2000).dropna()
ind_data_17 = ind_data_17.where(ind_data_17[ind_labels_17[5]] > 2000).dropna()
sg_data_14 = sg_data_14.where(sg_data_14[sg_labels_14[5]] > 2000).dropna()
sg_data_15 = sg_data_15.where(sg_data_15[sg_labels_15[5]] > 2000).dropna()
sg_data_16 = sg_data_16.where(sg_data_16[sg_labels_16[5]] > 2000).dropna()
sg_data_17 = sg_data_17.where(sg_data_17[sg_labels_17[5]] > 2000).dropna()

In [6]:
# Calculate transfers, premiums, costs, and reinsurance (if individual group) per member month
def calculate_pmm(df, df_labels, group):
    df["MM"] = df[df_labels[5]]
    df["Transfers"] = df[df_labels[2]]
    df["Enrollees"] = df["MM"] / 12
    df["TransfersPMM"] = df[df_labels[2]] / df[df_labels[5]]
    df["PremiumsPMM"] = abs(df[df_labels[3]] / df[df_labels[5]])
    df["CostsPMM"] = abs(df[df_labels[4]] / df[df_labels[5]])
    if group == "i":
        df["ReinsurancePMM"] = df[df_labels[6]] / df[df_labels[5]]
        df["ProfitsPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"] + df["ReinsurancePMM"]
        df["PremMinusCostsPlusReinsurancePMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["ReinsurancePMM"]
    elif group == "sg":
        df["ProfitsPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"]
    df["PremMinusCostsPMM"] = df["PremiumsPMM"] - df["CostsPMM"]
    df["PremMinusCostsPlusTransfersPMM"] = df["PremiumsPMM"] - df["CostsPMM"] + df["TransfersPMM"]
    df["SicknessPMM"] = df["CostsPMM"] - df["PremiumsPMM"]
    average_sickness = np.mean(df["SicknessPMM"])
    df["ExcessSicknessPMM"] = df["SicknessPMM"] - average_sickness
    return df

ind_data_14 = calculate_pmm(ind_data_14, ind_labels_14, "i")
ind_data_15 = calculate_pmm(ind_data_15, ind_labels_15, "i")
ind_data_16 = calculate_pmm(ind_data_16, ind_labels_16, "i")
ind_data_17 = calculate_pmm(ind_data_17, ind_labels_17, "sg")
sg_data_14 = calculate_pmm(sg_data_14, sg_labels_14, "sg")
sg_data_15 = calculate_pmm(sg_data_15, sg_labels_15, "sg")
sg_data_16 = calculate_pmm(sg_data_16, sg_labels_16, "sg")
sg_data_17 = calculate_pmm(sg_data_17, sg_labels_17, "sg")

In [7]:
# Combine the dataframes by left merging

ind_data = ind_data_14.merge(ind_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='outer').merge(ind_data_17, on='HIOS ID', how='outer', suffixes=('_16', '_17'))

In [8]:
# Do the same thing for small group

sg_data = sg_data_14.merge(sg_data_15, on='HIOS ID', how='outer', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='outer').merge(sg_data_17, on='HIOS ID', how='outer', suffixes=('_16', '_17'))

# Within-year analysis

- Mean, Variance, Min, Max

- Correlation coefficient analysis within each year

<a id='a1'></a>
### Mean, Variance, Min, Max

In [9]:
# Mean, Variance, Min, Max in a nice dataframe

df = pd.DataFrame(columns=["Year", "Group", "N", "Avg Costs", "Std Costs",
                           "Avg Premiums", "Std Premiums",
                           "Avg Abs Transfers", "Std Abs Transfers",
                           "Avg Transfers", "Std Transfers",
                           "Weighted Avg Transfers",
                           "Avg Profits", "Std Profits",
                           "Avg Reinsurance", "Std Reinsurance",
                          "Avg P-C", "Std P-C",
                          "Avg Excess Sickness", "Std Excess Sickness"])

years = [2014, 2015, 2016, 2017]
groups = ["Individual", "Small Group"]
ind_dfs = dict()
ind_dfs[2014] = ind_data_14
ind_dfs[2015] = ind_data_15
ind_dfs[2016] = ind_data_16
ind_dfs[2017] = ind_data_17
sg_dfs = dict()
sg_dfs[2014] = sg_data_14
sg_dfs[2015] = sg_data_15
sg_dfs[2016] = sg_data_16
sg_dfs[2017] = sg_data_17
data = dict()
data["Individual"] = ind_dfs
data["Small Group"] = sg_dfs
labels = ["CostsPMM", "PremiumsPMM", "TransfersPMM", "ProfitsPMM",
          "ReinsurancePMM", "PremMinusCostsPMM", "ExcessSicknessPMM"]

i = 0
for group in groups:
    for year in years:
        b = labels
        a = data[group][year]
        if group == "Individual" and year != 2017:
            df.loc[i] = [year, group, len(a),
                         round(np.mean(a[b[0]])), round(np.std(a[b[0]])),
                         round(np.mean(a[b[1]])), round(np.std(a[b[1]])),
                         round(np.mean(abs(a[b[2]]))), round(np.std(abs(a[b[2]]))),
                         round(np.mean(a[b[2]])), round(np.std(a[b[2]])),
                         round(np.average(a[b[2]], weights=a["MM"])),
                         round(np.mean(a[b[3]])), round(np.std(a[b[3]])),
                         round(np.mean(a[b[4]])), round(np.std(a[b[4]])),
                         round(np.mean(a[b[5]])), round(np.std(a[b[5]])),
                         round(np.mean(a[b[6]])), round(np.std(a[b[6]]))]
        else:
            df.loc[i] = [year, group, len(a),
                         round(np.mean(a[b[0]])), round(np.std(a[b[0]])),
                         round(np.mean(a[b[1]])), round(np.std(a[b[1]])),
                         round(np.mean(abs(a[b[2]]))), round(np.std(abs(a[b[2]]))),
                         round(np.mean(a[b[2]])), round(np.std(a[b[2]])),
                         round(np.average(a[b[2]], weights=a["MM"])),
                         round(np.mean(a[b[3]])), round(np.std(a[b[3]])),
                         np.nan, np.nan,
                         round(np.mean(a[b[5]])), round(np.std(a[b[5]])),
                         round(np.mean(a[b[6]])), round(np.std(a[b[6]]))]
        i = i + 1
df

,Year,Group,N,Avg Costs,Std Costs,Avg Premiums,Std Premiums,Avg Abs Transfers,Std Abs Transfers,Avg Transfers,Std Transfers,Weighted Avg Transfers,Avg Profits,Std Profits,Avg Reinsurance,Std Reinsurance,Avg P-C,Std P-C,Avg Excess Sickness,Std Excess Sickness
0,2014,Individual,403,366,183,314,92,42,57,16,69,-0.0,23,83,59,56,-52,144,0,144
1,2015,Individual,453,392,188,345,78,52,69,8,86,0.0,4,93,43,35,-47,151,0,151
2,2016,Individual,420,416,230,376,79,63,99,9,117,-0.0,-7,99,24,24,-40,190,0,190
3,2017,Individual,339,452,298,463,160,80,140,30,158,0.0,41,205,NaN,NaN,11,274,0,274
4,2014,Small Group,539,304,78,386,81,15,26,-7,29,-0.0,76,56,NaN,NaN,82,54,0,54
5,2015,Small Group,542,319,83,405,73,28,38,-13,45,0.0,73,76,NaN,NaN,86,66,0,66
6,2016,Small Group,475,340,84,422,70,29,39,-13,47,0.0,69,82,NaN,NaN,82,70,0,70
7,2017,Small Group,414,360,99,449,81,31,42,-11,52,-0.0,78,72,NaN,NaN,89,79,0,79


<a id='a2'></a>
### Correlation Coefficients within a year

In [10]:
# Correlation coefficient analysis within each year


df2 = pd.DataFrame(columns=["Year", "Group", "N", "Premium vs Transfers r", "Premium vs Transfers p",
                            "Transfers vs Costs r", "Transfers vs Costs p",
                            "Costs vs Premium r", "Costs vs Premium p",
                            "Profits vs Transfers r", "Profits vs Transfers p",
                            "Transfers vs Enrollees r", "Transfers vs Enrollees p"])

def within_year_analysis(df, year, group):
    r1 = pearsonr(df["PremiumsPMM"], df["TransfersPMM"])
    r2 = pearsonr(df["TransfersPMM"], df["CostsPMM"])
    r3 = pearsonr(df["CostsPMM"], df["PremiumsPMM"])
    r4 = pearsonr(df["ProfitsPMM"], df["TransfersPMM"])
    r5 = pearsonr(df["Transfers"], df["Enrollees"])
    #r5 = pearsonr(df["TransfersPMM"], df["Enrollees"])
    return [year, group, len(df), round(r1[0], 4), round(r1[1], 4),
            round(r2[0], 4), round(r2[1], 4), round(r3[0], 4),
            round(r3[1], 4), round(r4[0], 4), round(r4[1], 4),
            round(r5[0], 4), round(r5[1], 4)]
    
df2.loc[0] = within_year_analysis(ind_data_14, 2014, "individual")
df2.loc[1] = within_year_analysis(ind_data_15, 2015, "individual")
df2.loc[2] = within_year_analysis(ind_data_16, 2016, "individual")
df2.loc[3] = within_year_analysis(ind_data_17, 2017, "individual")
df2.loc[4] = within_year_analysis(sg_data_14, 2014, "small group")
df2.loc[5] = within_year_analysis(sg_data_15, 2015, "small group")
df2.loc[6] = within_year_analysis(sg_data_16, 2016, "small group")
df2.loc[7] = within_year_analysis(sg_data_17, 2017, "small group")

df2

,Year,Group,N,Premium vs Transfers r,Premium vs Transfers p,Transfers vs Costs r,Transfers vs Costs p,Costs vs Premium r,Costs vs Premium p,Profits vs Transfers r,Profits vs Transfers p,Transfers vs Enrollees r,Transfers vs Enrollees p
0,2014,individual,403,0.2898,0.0000,0.6666,0.0000,0.6300,0.0,0.0711,0.1542,0.0472,0.3450
1,2015,individual,453,0.3993,0.0000,0.7196,0.0000,0.6316,0.0,0.0435,0.3560,0.1726,0.0002
2,2016,individual,420,0.4221,0.0000,0.8319,0.0000,0.6372,0.0,-0.2219,0.0000,0.2655,0.0000
3,2017,individual,339,0.3314,0.0000,0.7928,0.0000,0.4131,0.0,-0.1223,0.0243,0.2705,0.0000
4,2014,small group,539,0.0098,0.8212,0.1524,0.0004,0.7664,0.0,0.3175,0.0000,0.2939,0.0000
5,2015,small group,542,0.1138,0.0080,0.1947,0.0000,0.6443,0.0,0.4942,0.0000,0.1725,0.0001
6,2016,small group,475,0.2429,0.0000,0.2509,0.0000,0.5983,0.0,0.5251,0.0000,0.1016,0.0269
7,2017,small group,414,0.1536,0.0017,0.4862,0.0000,0.6332,0.0,0.2215,0.0000,-0.0035,0.9435


# Between-year analysis

### Premiums - Costs ~ Transfers

In [11]:
# INDIVIDUAL GROUP Profits ~ Transfers

lhs = 'PremMinusCostsPMM'
rhs = 'TransfersPMM'

suffixes = ['_14', '_15', '_16', '_17']

for suffix in suffixes:
    print("Year %s" % (suffix))
    y_label = lhs + suffix
    x_label = rhs + suffix
    no_nulls = ind_data[[y_label, x_label]].dropna()
    X = no_nulls[[x_label]]
    y = no_nulls[y_label]
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit()
    print(est.summary())
    print("\n\n")

Year _14
                             OLS Regression Results                             
Dep. Variable:     PremMinusCostsPMM_14   R-squared:                       0.439
Model:                              OLS   Adj. R-squared:                  0.437
Method:                   Least Squares   F-statistic:                     313.3
Date:                  Sat, 28 Dec 2019   Prob (F-statistic):           3.22e-52
Time:                          12:36:21   Log-Likelihood:                -2458.5
No. Observations:                   403   AIC:                             4921.
Df Residuals:                       401   BIC:                             4929.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const        

In [12]:
# SMALL GROUP Profits ~ Transfers

lhs = 'PremMinusCostsPMM'
rhs = 'TransfersPMM'

suffixes = ['_14', '_15', '_16', '_17']

for suffix in suffixes:
    print("Year %s" % (suffix))
    y_label = lhs + suffix
    x_label = rhs + suffix
    no_nulls = sg_data[[y_label, x_label]].dropna()
    X = no_nulls[[x_label]]
    y = no_nulls[y_label]
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit()
    print(est.summary())
    print("\n\n")

Year _14
                             OLS Regression Results                             
Dep. Variable:     PremMinusCostsPMM_14   R-squared:                       0.041
Model:                              OLS   Adj. R-squared:                  0.039
Method:                   Least Squares   F-statistic:                     23.01
Date:                  Sat, 28 Dec 2019   Prob (F-statistic):           2.09e-06
Time:                          12:36:21   Log-Likelihood:                -2908.4
No. Observations:                   539   AIC:                             5821.
Df Residuals:                       537   BIC:                             5829.
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const        

In [13]:
# INDIVIDUAL GROUP Profits (incl. Reinsurance) ~ Transfers

lhs = 'PremMinusCostsPlusReinsurancePMM'
rhs = 'TransfersPMM'

suffixes = ['_14', '_15', '']

for suffix in suffixes:
    if suffix == '':
        print("Year %s" % ('_16'))
        x_label = rhs + '_16'
        y_label = lhs
    else:
        x_label = rhs + suffix
        y_label = lhs + suffix
    no_nulls = ind_data[[y_label, x_label]].dropna()
    X = no_nulls[[x_label]]
    y = no_nulls[y_label]
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit()
    print(est.summary())
    print("\n\n")

                                     OLS Regression Results                                    
Dep. Variable:     PremMinusCostsPlusReinsurancePMM_14   R-squared:                       0.364
Model:                                             OLS   Adj. R-squared:                  0.363
Method:                                  Least Squares   F-statistic:                     229.8
Date:                                 Sat, 28 Dec 2019   Prob (F-statistic):           2.33e-41
Time:                                         12:36:21   Log-Likelihood:                -2351.4
No. Observations:                                  403   AIC:                             4707.
Df Residuals:                                      401   BIC:                             4715.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                      coef    std err   

<a id='c1'></a>
## Effect of losses in 2014 and 2015 on exiting the market in 2016

In [14]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

ind_data_exit = ind_data_14.merge(ind_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='left')
ind_data_exit["LossQuintile"] = pd.qcut(ind_data_exit['ProfitsPMM_14'], 5, labels=False)
ind_data_exit["ExitedMarketIn2016"] = np.where(ind_data_exit[['TransfersPMM']].isnull(), 1, 0)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile'])], axis=1)
ind_data_exit.columns = ind_data_exit.columns.astype(str)

In [15]:
# Looking at the effect of losses in 2014 and 2015 on exiting the market in 2016

sg_data_exit = sg_data_14.merge(sg_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='left')
sg_data_exit["LossQuintile"] = pd.qcut(sg_data_exit['ProfitsPMM_14'], 5, labels=False)
sg_data_exit["ExitedMarketIn2016"] = np.where(sg_data_exit[['TransfersPMM']].isnull(), 1, 0)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile'])], axis=1)
sg_data_exit.columns = sg_data_exit.columns.astype(str)

<a id='c2'></a>
### ExitedMarketIn2016 = Loss 2014 Q1 + Loss 2014 Q5

In [16]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = ind_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.578562
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  403
Model:                          Logit   Df Residuals:                      400
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.01299
Time:                        12:36:21   Log-Likelihood:                -233.16
converged:                       True   LL-Null:                       -236.23
                                        LLR p-value:                   0.04654
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9134      0.143     -6.410      0.000      -1.193      -0.634
0              0.2202      0.275      0.800      0.424      -0.320       0.760
4             -0.6522      0.327     -1.997      0.046      -1.292      -0.012
==============================================================================
"""

In [17]:
# ExitedMarketIn 2016 = Loss 2014 Q1 + Loss 2014 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = sg_data_exit[['0', '4', 'ExitedMarketIn2016']].dropna()
X = no_nulls[['0', '4']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.469176
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  539
Model:                          Logit   Df Residuals:                      536
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.05852
Time:                        12:36:21   Log-Likelihood:                -252.89
converged:                       True   LL-Null:                       -268.60
                                        LLR p-value:                 1.491e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9566      0.169    -11.583      0.000      -2.288      -1.625
0              1.4259      0.261      5.459      0.000       0.914       1.938
4              0.8580      0.279      3.074      0.002       0.311       1.405
==============================================================================
"""

<a id='c3'></a>
### ExitedMarketIn2016 = Profits 2014 + Profits 2015

In [18]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = ind_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ExitedMarketIn2016']].dropna()
print("Individual Group")
print("Loss 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Loss 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Individual Group
Loss 14 Avg: 22.879370, Range: 677.705621
Loss 15 Avg: 4.691754, Range: 1141.735553
Optimization terminated successfully.
         Current function value: 0.522292
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  376
Model:                          Logit   Df Residuals:                      373
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.02871
Time:                        12:36:22   Log-Likelihood:                -196.38
converged:                       True   LL-Null:                       -202.19
                                        LLR p-value:                  0.003015
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.2515      0.126     -9.909      0.000      -1.499      -1.004
ProfitsPMM_14_Norm    -3.6449      1.262     -2.888      0.004      -6.118      -1.171
ProfitsPMM_15_Norm     0.2177      1.692      0.129      0.898      -3.099       3.535
======================================================================================
"""

In [19]:
# ExitedMarketIn 2016 = Loss 2014 + Loss 2015

no_nulls = sg_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ExitedMarketIn2016']].dropna()
print("Small Group")
print("Loss 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Loss 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm']]
y = no_nulls['ExitedMarketIn2016']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Small Group
Loss 14 Avg: 76.585056, Range: 518.041756
Loss 15 Avg: 78.535117, Range: 708.086313
Optimization terminated successfully.
         Current function value: 0.389014
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2016   No. Observations:                  499
Model:                          Logit   Df Residuals:                      496
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.02292
Time:                        12:36:22   Log-Likelihood:                -194.12
converged:                       True   LL-Null:                       -198.67
                                        LLR p-value:                   0.01053
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.8998      0.136    -13.950      0.000      -2.167      -1.633
ProfitsPMM_14_Norm    -2.7060      1.584     -1.708      0.088      -5.811       0.399
ProfitsPMM_15_Norm    -1.5096      1.468     -1.028      0.304      -4.386       1.367
======================================================================================
"""

Effect of Prior years' Losses on Exiting the Market in 2017

In [20]:
# Looking at the effect of losses in 2014 - 2016 on exiting the market in 2017

ind_data_exit = ind_data_14.merge(ind_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(ind_data_16, on='HIOS ID', how='left').merge(ind_data_17, on='HIOS ID', how='left', suffixes=('_16', '_17'))
ind_data_exit["LossQuintile_14"] = pd.qcut(ind_data_exit['ProfitsPMM_14'], 5, labels=False)
ind_data_exit["LossQuintile_15"] = pd.qcut(ind_data_exit['ProfitsPMM_15'], 5, labels=False)
ind_data_exit["LossQuintile_16"] = pd.qcut(ind_data_exit['ProfitsPMM_16'], 5, labels=False)
ind_data_exit["ExitedMarketIn2017"] = np.where(ind_data_exit[['TransfersPMM_17']].isnull(), 1, 0)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile_14'], prefix='LQ_14_')], axis=1)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile_15'], prefix='LQ_15_')], axis=1)
ind_data_exit = pd.concat([ind_data_exit, pd.get_dummies(ind_data_exit['LossQuintile_16'], prefix='LQ_16_')], axis=1)
ind_data_exit.columns = ind_data_exit.columns.astype(str)


no_nulls = ind_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ProfitsPMM_16', 'ExitedMarketIn2017']].dropna()
print("Individual Group")
print("Profits 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Profits 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
print("Profits 16 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_16"].mean(), no_nulls["ProfitsPMM_16"].max() - no_nulls["ProfitsPMM_16"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
no_nulls["ProfitsPMM_16_Norm"] = (no_nulls["ProfitsPMM_16"] - no_nulls["ProfitsPMM_16"].mean()) / (no_nulls["ProfitsPMM_16"].max() - no_nulls["ProfitsPMM_16"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm', 'ProfitsPMM_16_Norm']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Individual Group
Profits 14 Avg: 30.478771, Range: 502.422763
Profits 15 Avg: 9.161246, Range: 1141.735553
Profits 16 Avg: -5.524914, Range: 1662.364406
Optimization terminated successfully.
         Current function value: 0.448986
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  290
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                            3
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.07563
Time:                        12:36:22   Log-Likelihood:                -130.21
converged:                       True   LL-Null:                       -140.86
                                        LLR p-value:                 9.088e-05
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.5825      0.165     -9.590      0.000      -1.906      -1.259
ProfitsPMM_14_Norm    -2.1470      1.262     -1.701      0.089      -4.621       0.327
ProfitsPMM_15_Norm    -9.4440      3.001     -3.147      0.002     -15.326      -3.562
ProfitsPMM_16_Norm     7.9523      3.362      2.365      0.018       1.362      14.542
======================================================================================
"""

In [21]:
no_nulls = ind_data_exit[['TransfersPMM_14', 'TransfersPMM_15', 'TransfersPMM_16', 'ExitedMarketIn2017']].dropna()
no_nulls["TransfersPMM_14_Norm"] = (no_nulls["TransfersPMM_14"] - no_nulls["TransfersPMM_14"].mean()) / (no_nulls["TransfersPMM_14"].max() - no_nulls["TransfersPMM_14"].min())
no_nulls["TransfersPMM_15_Norm"] = (no_nulls["TransfersPMM_15"] - no_nulls["TransfersPMM_15"].mean()) / (no_nulls["TransfersPMM_15"].max() - no_nulls["TransfersPMM_15"].min())
no_nulls["TransfersPMM_16_Norm"] = (no_nulls["TransfersPMM_16"] - no_nulls["TransfersPMM_16"].mean()) / (no_nulls["TransfersPMM_16"].max() - no_nulls["TransfersPMM_16"].min())
X = no_nulls[['TransfersPMM_14_Norm', 'TransfersPMM_15_Norm', 'TransfersPMM_16_Norm']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.480027
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  290
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                            3
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.01173
Time:                        12:36:22   Log-Likelihood:                -139.21
converged:                       True   LL-Null:                       -140.86
                                        LLR p-value:                    0.3471
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -1.4751      0.153     -9.659      0.000      -1.774      -1.176
TransfersPMM_14_Norm     1.5589      1.657      0.941      0.347      -1.689       4.807
TransfersPMM_15_Norm    -7.3805      4.237     -1.742      0.081     -15.684       0.923
TransfersPMM_16_Norm     4.1028      3.648      1.125      0.261      -3.046      11.252
========================================================================================
"""

In [22]:
# ExitedMarketIn 2017 = Loss 2015 Q1 + Loss 2015 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = ind_data_exit[['LQ_15__0.0', 'LQ_15__4.0', 'ExitedMarketIn2017']].dropna()
X = no_nulls[['LQ_15__0.0', 'LQ_15__4.0']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.654706
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  403
Model:                          Logit   Df Residuals:                      400
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.02691
Time:                        12:36:22   Log-Likelihood:                -263.85
converged:                       True   LL-Null:                       -271.14
                                        LLR p-value:                 0.0006784
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3857      0.128     -3.005      0.003      -0.637      -0.134
LQ_15__0.0     0.5439      0.264      2.064      0.039       0.027       1.060
LQ_15__4.0    -0.7670      0.299     -2.563      0.010      -1.354      -0.180
==============================================================================
"""

In [23]:
# Looking at the effect of losses in 2014 - 2016 on exiting the market in 2017

sg_data_exit = sg_data_14.merge(sg_data_15, on='HIOS ID', how='left', suffixes=('_14', '_15'))\
.merge(sg_data_16, on='HIOS ID', how='left').merge(sg_data_17, on='HIOS ID', how='left', suffixes=('_16', '_17'))
sg_data_exit["LossQuintile_14"] = pd.qcut(sg_data_exit['ProfitsPMM_14'], 5, labels=False)
sg_data_exit["LossQuintile_15"] = pd.qcut(sg_data_exit['ProfitsPMM_15'], 5, labels=False)
sg_data_exit["LossQuintile_16"] = pd.qcut(sg_data_exit['ProfitsPMM_16'], 5, labels=False)
sg_data_exit["ExitedMarketIn2017"] = np.where(sg_data_exit[['TransfersPMM_17']].isnull(), 1, 0)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile_14'], prefix='LQ_14_')], axis=1)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile_15'], prefix='LQ_15_')], axis=1)
sg_data_exit = pd.concat([sg_data_exit, pd.get_dummies(sg_data_exit['LossQuintile_16'], prefix='LQ_16_')], axis=1)
sg_data_exit.columns = sg_data_exit.columns.astype(str)


no_nulls = sg_data_exit[['ProfitsPMM_14', 'ProfitsPMM_15', 'ProfitsPMM_16', 'ExitedMarketIn2017']].dropna()
print("Small Group")
print("Profits 14 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_14"].mean(), no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min()))
print("Profits 15 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_15"].mean(), no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min()))
print("Profits 16 Avg: %f, Range: %f" % (no_nulls["ProfitsPMM_16"].mean(), no_nulls["ProfitsPMM_16"].max() - no_nulls["ProfitsPMM_16"].min()))
no_nulls["ProfitsPMM_14_Norm"] = (no_nulls["ProfitsPMM_14"] - no_nulls["ProfitsPMM_14"].mean()) / (no_nulls["ProfitsPMM_14"].max() - no_nulls["ProfitsPMM_14"].min())
no_nulls["ProfitsPMM_15_Norm"] = (no_nulls["ProfitsPMM_15"] - no_nulls["ProfitsPMM_15"].mean()) / (no_nulls["ProfitsPMM_15"].max() - no_nulls["ProfitsPMM_15"].min())
no_nulls["ProfitsPMM_16_Norm"] = (no_nulls["ProfitsPMM_16"] - no_nulls["ProfitsPMM_16"].mean()) / (no_nulls["ProfitsPMM_16"].max() - no_nulls["ProfitsPMM_16"].min())
X = no_nulls[['ProfitsPMM_14_Norm', 'ProfitsPMM_15_Norm', 'ProfitsPMM_16_Norm']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Small Group
Profits 14 Avg: 79.281480, Range: 518.041756
Profits 15 Avg: 81.778846, Range: 456.430417
Profits 16 Avg: 77.754121, Range: 584.560847
Optimization terminated successfully.
         Current function value: 0.423503
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  431
Model:                          Logit   Df Residuals:                      427
Method:                           MLE   Df Model:                            3
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.01979
Time:                        12:36:22   Log-Likelihood:                -182.53
converged:                       True   LL-Null:                       -186.21
                                        LLR p-value:                   0.06103
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.7345      0.138    -12.613      0.000      -2.004      -1.465
ProfitsPMM_14_Norm     2.2711      1.616      1.405      0.160      -0.896       5.438
ProfitsPMM_15_Norm     0.2174      1.541      0.141      0.888      -2.803       3.238
ProfitsPMM_16_Norm    -3.7124      1.605     -2.313      0.021      -6.858      -0.567
======================================================================================
"""

In [24]:
no_nulls = sg_data_exit[['TransfersPMM_14', 'TransfersPMM_15', 'TransfersPMM_16', 'ExitedMarketIn2017']].dropna()
no_nulls["TransfersPMM_14_Norm"] = (no_nulls["TransfersPMM_14"] - no_nulls["TransfersPMM_14"].mean()) / (no_nulls["TransfersPMM_14"].max() - no_nulls["TransfersPMM_14"].min())
no_nulls["TransfersPMM_15_Norm"] = (no_nulls["TransfersPMM_15"] - no_nulls["TransfersPMM_15"].mean()) / (no_nulls["TransfersPMM_15"].max() - no_nulls["TransfersPMM_15"].min())
no_nulls["TransfersPMM_16_Norm"] = (no_nulls["TransfersPMM_16"] - no_nulls["TransfersPMM_16"].mean()) / (no_nulls["TransfersPMM_16"].max() - no_nulls["TransfersPMM_16"].min())
X = no_nulls[['TransfersPMM_14_Norm', 'TransfersPMM_15_Norm', 'TransfersPMM_16_Norm']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.424320
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  431
Model:                          Logit   Df Residuals:                      427
Method:                           MLE   Df Model:                            3
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                 0.01789
Time:                        12:36:22   Log-Likelihood:                -182.88
converged:                       True   LL-Null:                       -186.21
                                        LLR p-value:                   0.08339
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -1.7214      0.136    -12.686      0.000      -1.987      -1.455
TransfersPMM_14_Norm    -1.7589      1.756     -1.002      0.316      -5.200       1.682
TransfersPMM_15_Norm     1.0315      2.355      0.438      0.661      -3.583       5.646
TransfersPMM_16_Norm    -3.1299      2.133     -1.467      0.142      -7.311       1.051
========================================================================================
"""

In [25]:
# ExitedMarketIn 2017 = Loss 2015 Q1 + Loss 2015 Q5

# "0" = you are losing money (loss is negative, which means Costs > Premiums + Transfers + Reinsurance)
# "4" = you are making money (loss is positive, which means Costs < Premiums + Transfers + Reinsurance)

no_nulls = sg_data_exit[['LQ_15__0.0', 'LQ_15__4.0', 'ExitedMarketIn2017']].dropna()
X = no_nulls[['LQ_15__0.0', 'LQ_15__4.0']]
y = no_nulls['ExitedMarketIn2017']
X = sm.add_constant(X)
est = sm.Logit(y, X).fit()
est.summary()

Optimization terminated successfully.
         Current function value: 0.626728
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     ExitedMarketIn2017   No. Observations:                  539
Model:                          Logit   Df Residuals:                      536
Method:                           MLE   Df Model:                            2
Date:                Sat, 28 Dec 2019   Pseudo R-squ.:                0.001380
Time:                        12:36:22   Log-Likelihood:                -337.81
converged:                       True   LL-Null:                       -338.27
                                        LLR p-value:                    0.6269
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7739      0.117     -6.623      0.000      -1.003      -0.545
LQ_15__0.0     0.1985      0.239      0.831      0.406      -0.270       0.667
LQ_15__4.0    -0.0734      0.248     -0.296      0.767      -0.559       0.412
==============================================================================
"""